# Computer lab :

Group Members : Ayoub Marzoug - Adnane El Bouhali

## Let’s reverse-engineer the data center SD-TSIA 211

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Loading data
data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

### Least Squares

**Question 3.1 : Show that if $Aw=b$, then $\displaystyle y(t)=\frac{w_1^{\top} \tilde{x}(t)+w_0}{w_2^{\top} \tilde{x}(t)+1}.$**

If $Aw=b$, then for all $t$ :
$$ (Aw)_t = \tilde{x}(t)^{T}w_1 + w_0 − y(t)\times \tilde{x}(t)^{T}w_2 = y(t)$$

$$y(t)\big(w_2^{T}\tilde{x}(t)+ 1\big) = w_1^{T}\tilde{x}(t) + w_0$$

$$y(t) = \frac{w_1^{T}\tilde{x}(t) + w_0}{w_2^{T}\tilde{x}(t)+ 1}$$

**Question 3.2 : Solve this least squares problem using the function `numpy.linalg.lstsq`.**

In [3]:
# Constructing matrices for min_w ||A w - b||_2**2

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

In [4]:
wmin = np.linalg.lstsq(A, b, rcond=None)[0]

**Question 3.3 : Evaluate the quality of the solution found on the test set.**

In [5]:
# Constructing matrices for the test set

M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]

In [6]:
# Predicting the values
y_pred = np.dot(A_test, wmin)

# Calculating the mean squared error
mse_train = np.mean((np.dot(A, wmin) - b)**2)
mse_test = np.mean((y_pred - b_test)**2)
print(mse_train, mse_test)

9.769461864685916e-28 780.898479352361


**Question 3.4 : In order to improve the generalization power of the model, we consider a $\mathscr l_2$ regularization : $\min_w\frac 12\|Aw − b\|^2 +\frac λ2 \|w\|^2$ where $λ = 100$. Solve this problem and compare the test mean square error with the unregularized one.**

In [7]:
# Augment A and b
λ = 100
n_col = A.shape[1]
A_augmented = np.vstack([A, np.sqrt(λ) * np.identity(n_col)])
b_augmented = np.vstack([b.reshape(-1, 1), np.zeros((n_col, 1))])

# Solve the modified least squares problem
wridge = np.linalg.lstsq(A_augmented, b_augmented, rcond=None)[0]

In [8]:
# Predicting the values
y_pred = np.dot(A_test, wridge)

# Calculating the mean squared error
mse_train = np.mean((np.dot(A, wridge) - b)**2)
mse_test = np.mean((y_pred - b_test)**2)
print(mse_train, mse_test)

9.171430604344522 316.9616528129709


**Question 3.5 : Calculate the gradient of $f_1 : w\mapsto \frac 12 \|Aw − b\|^2 +\frac λ2 \|w\|^2$ . Is the function convex ?**

For $w \in \mathbb{R}^{2d+1}$, we have :
$$
f_1(w) = \frac{1}{2} (Aw - b)^{T}(Aw-b) + \frac{\lambda}{2} w^{T}w
$$
Thus :
$$
\nabla f_1(w) = \frac{1}{2}\big(A^{T}(Aw-b)+A^{T}(Aw-b)\big) + \frac{\lambda}2\big(w + w \big)
\\ = A^{T}(Aw-b) + \lambda w
$$
$$
\nabla^2 f_1(w) = A^{T}A + \lambda I
$$
This function is convex, since the $\lambda I$ is positive definite for any $\lambda>0$, and $A^𝑇A$ is positive semi-definite for any $A$. The sum of a positive definite and positive semi-definite matrix is positive definite, QED.

**Question 3.6 : Implement gradient descent to minimize $f_1$. What step size are you choosing? How many iterations are needed to get $w_k$ such that $\|∇f (w_k)\| ≤ 1$ ?**

In [9]:
def f1(A, b, w, λ):
    term1 = np.linalg.norm(A @ w - b)**2 / 2
    term2 = λ * np.linalg.norm(w)**2 / 2
    return term1 + term2

def gradient_f1(A, b, w, λ):
    A = np.array(A)
    return A.T @ (A @ w - b) + λ * w

def gradient_descent(A, b, λ, initial_w, alpha):
    w = initial_w
    num_iterations = 0
    while np.linalg.norm(gradient_f1(A, b, w, λ)) > 1:
        gradient = gradient_f1(A, b, w, λ)
        w = w - alpha * gradient
        num_iterations += 1
    return num_iterations

In [10]:
def grid_search(A, b, λ, initial_w, alpha_values):
    best_alpha = None
    min_iterations = float('inf')

    for alpha in alpha_values:
        iterations = gradient_descent(A, b, λ, initial_w, alpha)
        if iterations < min_iterations:
            min_iterations = iterations
            best_alpha = alpha

    return best_alpha, min_iterations

In [11]:
initial_w = np.zeros(1785)
alpha_values = [0.01, 0.1, 0.5, 1.0, 2.0]

best_alpha, min_iterations = grid_search(A, b, λ, initial_w, alpha_values)

print(f"Best Alpha: {best_alpha}")
print(f"Minimum Iterations: {min_iterations}")

/var/folders/1v/y6b4xp493972fjd99f2w41j80000gn/T/ipykernel_10649/335825371.py:8: RuntimeWarning: overflow encountered in matmul
  return A.T @ (A @ w - b) + λ * w
/var/folders/1v/y6b4xp493972fjd99f2w41j80000gn/T/ipykernel_10649/335825371.py:8: RuntimeWarning: invalid value encountered in matmul
  return A.T @ (A @ w - b) + λ * w
/var/folders/1v/y6b4xp493972fjd99f2w41j80000gn/T/ipykernel_10649/335825371.py:8: RuntimeWarning: overflow encountered in multiply
  return A.T @ (A @ w - b) + λ * w


Best Alpha: 2.0
Minimum Iterations: 45


**Question 3.7 : (Bonus question) Implement the conjugate gradient method. Compare the convergence rate with the previous algorithm.**

In [12]:
import numpy as np

def conjugate_gradient(A, b, x0=None, tol=1e-6, max_iter=1000):
    n = len(b)
    
    if x0 is None:
        x0 = np.zeros(n)
    
    x = x0
    r = b - np.dot(A, x)
    p = r
    rsold = np.dot(r, r)

    for i in range(max_iter):
        Ap = np.dot(A.T, p)  # Transpose A before multiplication
        alpha = rsold / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * np.dot(A, p)
        rsnew = np.dot(r, r)
        if np.sqrt(rsnew) < tol:
            break
        beta = rsnew / rsold
        p = r + beta * p
        rsold = rsnew

    return x

### Regularization for a sparse model

**Question 4.1 : Write the objective function as $F_2 = f_2 + g_2$ where $f_2$ is differentiable and the proximal operator of $g_2$ is easy to compute. Recall the formula for $prox_{g2}$. Calculate the gradient of $f_2$.**

The objective function is : $F_2(w) = \frac{1}{2}\|Aw-b\|^2 + \lambda \|w\|_1$.

We have $f_2(w) = \frac{1}{2}\|Aw-b\|^2$ and $g_2(w) = \lambda \|w\|_1$.

The proximal operator of $g_2$ is easy to compute since it is the soft thresholding operator : $prox_{g2}(w) = S_{\lambda}(w)$.

The gradient of $f_2$ is :

$$
\nabla f_2(w) = A^{T}(Aw-b)
$$

**Question 4.2 : Code the proximal gradient method. Here, we will take $λ = 200$. What stopping test do you suggest ?**

In [13]:
def soft_thresholding(x, threshold):
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

def proximal_gradient(A, b, λ, initial_w, alpha, max_iter=1000, tol=1e-6):
    w = initial_w
    for i in range(max_iter):
        gradient = A.T @ (A @ w - b)
        w_new = soft_thresholding(w - alpha * gradient, λ * alpha)
        if np.linalg.norm(w_new - w) < tol:
            break
        w = w_new
    return w

In [28]:
wlasso = proximal_gradient(A, b, 200, initial_w, alpha=0.0001, max_iter=100, tol=1e-6)

**Question 4.3 : We may try to accelerate the algorithm using line search. Compare the speed of the algorithm with fixed step size and with line search.**

In [15]:
def line_search_proximal_gradient(A, b, λ, initial_w, max_iter=1000, tol=1e-6):
    w = initial_w
    alpha = 1.0
    for i in range(max_iter):
        gradient = A.T @ (A @ w - b)
        w_new = soft_thresholding(w - alpha * gradient, λ * alpha)
        obj_new = 0.5 * np.linalg.norm(A @ w_new - b)**2 + λ * np.linalg.norm(w_new, 1)
        obj_diff = obj_new - (0.5 * np.linalg.norm(A @ w - b)**2 + λ * np.linalg.norm(w, 1))
        while obj_diff > 0:
            alpha *= 0.5
            w_new = soft_thresholding(w - alpha * gradient, λ * alpha)
            obj_new = 0.5 * np.linalg.norm(A @ w_new - b)**2 + λ * np.linalg.norm(w_new, 1)
            obj_diff = obj_new - (0.5 * np.linalg.norm(A @ w - b)**2 + λ * np.linalg.norm(w, 1))
        if np.linalg.norm(w_new - w) < tol:
            break
        w = w_new
    return w

In [16]:
import time

# Fixed step size
start_time_fixed = time.time()
w_fixed = proximal_gradient(A, b, 200, initial_w, alpha=0.1, max_iter=1000, tol=1e-6)
end_time_fixed = time.time()
execution_time_fixed = end_time_fixed - start_time_fixed

# Line search
start_time_line_search = time.time()
w_line_search = line_search_proximal_gradient(A, b, 200, initial_w, max_iter=1000, tol=1e-6)
end_time_line_search = time.time()
execution_time_line_search = end_time_line_search - start_time_line_search

execution_time_fixed, execution_time_line_search

/var/folders/1v/y6b4xp493972fjd99f2w41j80000gn/T/ipykernel_10649/3118583458.py:7: RuntimeWarning: overflow encountered in matmul
  gradient = A.T @ (A @ w - b)
/var/folders/1v/y6b4xp493972fjd99f2w41j80000gn/T/ipykernel_10649/3118583458.py:7: RuntimeWarning: invalid value encountered in matmul
  gradient = A.T @ (A @ w - b)


(0.7472579479217529, 1.7163522243499756)

### Comparison

**Question 6.1 : Compare the solutions obtained by the two types of regularization.**

In [30]:
# Calculate MSE for unregularized solution
mse_ridge = np.mean((np.dot(A_test, wridge) - b_test)**2)

# Calculate MSE for regularized solution
mse_lasso = np.mean((np.dot(A_test, wlasso) - b_test)**2)

mse_ridge, mse_lasso

/var/folders/1v/y6b4xp493972fjd99f2w41j80000gn/T/ipykernel_10649/2798786856.py:5: RuntimeWarning: overflow encountered in square
  mse_lasso = np.mean((np.dot(A_test, wlasso) - b_test)**2)


(316.9616528129709, inf)